# Overview of MaskedTensors

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pytorch/maskedtensor/blob/main/docs/source/notebooks/overview.ipynb)

In [1]:
import torch
import numpy as np
from maskedtensor import masked_tensor
from maskedtensor import as_masked_tensor

## Basic masking semantics

### MaskedTensor vs NumPy's MaskedArray semantics

In [2]:
# First example of addition
data = torch.arange(5.)
mask = torch.tensor([True, True, False, True, False])
m0 = masked_tensor(data, mask)
m1 = masked_tensor(data, ~mask)

print(m0)
print(m1)
print(torch.cos(m0))
print(m0 + m0)

try:
  # For now the masks must match. We treat them like shapes.
  # We can relax this later on, but should have a good reason for it.
  # We'll revisit this once we have reductions.
  print(m0 + m1)
except ValueError as e:
  print(e)

masked_tensor(
  [  0.0000,   1.0000,       --,   3.0000,       --]
)
masked_tensor(
  [      --,       --,   2.0000,       --,   4.0000]
)
masked_tensor(
  [  1.0000,   0.5403,       --,  -0.9900,       --]
)
masked_tensor(
  [  0.0000,   2.0000,       --,   6.0000,       --]
)
Input masks must match. If you need support for this, please open an issue on Github.


NumPy's MaskedArray implements intersection semantics here. If one of two elements are masked out the resulting element will be masked out as well. Note that MaskedArray's factory function inverts the mask (similar to torch.nn.MHA). For MaskedTensor we'd apply the logical_and operator to both masks during a binary operation to get the semantics NumPy has. Since NumPy stores the inverted mask they [apply the logical_or operator](https://github.com/numpy/numpy/blob/68299575d8595d904aff6f28e12d21bf6428a4ba/numpy/ma/core.py#L1016-L1024). But to repeat this point we suggest to not support addition between MaskedTensors with masks that don't match. See the section on reductions for why we should have good reasons for this.

In [3]:
npm0 = np.ma.masked_array(data.numpy(), (~mask).numpy())
npm1 = np.ma.masked_array(data.numpy(), (mask).numpy())
print("npm0:       ", npm0)
print("npm1:       ", npm1)
print("npm0 + npm1:", npm0 + npm1)

npm0:        [0.0 1.0 -- 3.0 --]
npm1:        [-- -- 2.0 -- 4.0]
npm0 + npm1: [-- -- -- -- --]


MaskedTensor also supports these semantics by giving access to the masks and conveniently converting a MaskedTensor to a Tensor with masked values filled in with a particular value.

NumPy of course has the opportunity to avoid addition altogether in this case by check whether any results are not masked, but [chooses not to](https://github.com/numpy/numpy/blob/68299575d8595d904aff6f28e12d21bf6428a4ba/numpy/ma/core.py#L1013). Presumably it's more expensive to allreduce the mask every time to avoid the binary addition of the data in this case.

In [4]:
m0t = m0.to_tensor(0)
m1t = m1.to_tensor(0)

m2t = masked_tensor(m0t + m1t, m0.mask() & m1.mask())
print(m0t)
print(m1t)
print(m2t)

tensor([0., 1., 0., 3., 0.])
tensor([0., 0., 2., 0., 4.])
masked_tensor(
  [      --,       --,       --,       --,       --]
)


### MaskedTensor reduction semantics

Example of printing a 2d MaskedTensor and setup for reductions below

In [5]:
data = torch.randn(8, 3).mul(10).int().float()
mask = torch.randint(2, (8, 3), dtype=torch.bool)
print(data)
print(mask)
m = masked_tensor(data, mask)
print(m)

tensor([[  6.,  10.,   9.],
        [  9.,  19.,   0.],
        [  4., -10., -16.],
        [  7., -10.,  -2.],
        [  0.,   9.,  14.],
        [  4.,   5.,   7.],
        [  6., -10.,  12.],
        [-16.,  -6.,  12.]])
tensor([[ True,  True, False],
        [ True, False, False],
        [ True, False, False],
        [ True,  True,  True],
        [ True,  True, False],
        [False,  True,  True],
        [False,  True, False],
        [False, False,  True]])
masked_tensor(
  [
    [  6.0000,  10.0000,       --],
    [  9.0000,       --,       --],
    [  4.0000,       --,       --],
    [  7.0000, -10.0000,  -2.0000],
    [  0.0000,   9.0000,       --],
    [      --,   5.0000,   7.0000],
    [      --, -10.0000,       --],
    [      --,       --,  12.0000]
  ]
)


Reduction semantics based on https://github.com/pytorch/rfcs/pull/27

In [6]:
print("sum:", torch.sum(m, 1))
print("mean:", torch.mean(m, 1))
print("prod:", torch.prod(m, 1))
print("min:", torch.amin(m, 1))
print("max:", torch.amax(m, 1))

sum: masked_tensor(
  [ 16.0000,   9.0000,   4.0000,  -5.0000,   9.0000,  12.0000, -10.0000,  12.0000]
)
mean: masked_tensor(
  [  8.0000,   9.0000,   4.0000,  -1.6667,   4.5000,   6.0000, -10.0000,  12.0000]
)
prod: masked_tensor(
  [ 60.0000,   9.0000,   4.0000, 140.0000,   0.0000,  35.0000, -10.0000,  12.0000]
)
min: masked_tensor(
  [  6.0000,   9.0000,   4.0000, -10.0000,   0.0000,   5.0000, -10.0000,  12.0000]
)
max: masked_tensor(
  [ 10.0000,   9.0000,   4.0000,   7.0000,   9.0000,   7.0000, -10.0000,  12.0000]
)


Now that we have reductions, let's revisit as to why we'll probably want to have a good reason to allow addition of MaskedTensors with different masks.

In [7]:
data0 = torch.arange(10.).reshape(2, 5)
data1 = torch.arange(10.).reshape(2, 5) + 10
mask0 = torch.tensor([[True, True, False, False, False], [False, False, False, True, True]])
mask1 = torch.tensor([[False, False, False, True, True], [True, True, False, False, False]])

npm0 = np.ma.masked_array(data0.numpy(), (mask0).numpy())
npm1 = np.ma.masked_array(data1.numpy(), (mask1).numpy())
print("\nnpm0:\n", npm0)
print("\nnpm1:\n", npm1)
print("\n(npm0 + npm1).sum(0):\n", (npm0 + npm1).sum(0))
print("\nnpm0.sum(0) + npm1.sum(0):\n", (npm0.sum(0) + npm1.sum(0)))
print("\n(data0 + data1).sum(0):\n", (data0 + data1).sum(0))
print("\n(data0 + data1).sum(0):\n", (data0.sum(0) + data1.sum(0)))


npm0:
 [[-- -- 2.0 3.0 4.0]
 [5.0 6.0 7.0 -- --]]

npm1:
 [[10.0 11.0 12.0 -- --]
 [-- -- 17.0 18.0 19.0]]

(npm0 + npm1).sum(0):
 [-- -- 38.0 -- --]

npm0.sum(0) + npm1.sum(0):
 [15.0 17.0 38.0 21.0 23.0]

(data0 + data1).sum(0):
 tensor([30., 34., 38., 42., 46.])

(data0 + data1).sum(0):
 tensor([30., 34., 38., 42., 46.])


Sum and addition should be associative. However with NumPy's semantics we allow them not to be. Instead of allowing these semantics, at least in the case of addition and sum, we could ask the user to fill the MaskedTensor's undefined elements with 0 values or as in the MaskedTensor addition examples above be very specific about the semantics used.

While it's obviously possible to support this, we think we should cover other operators first and really make sure we can't avoid this behavior via other means.

### Indexing and Advanced Indexing

In [8]:
data = torch.randn(4, 5, 3).mul(5).float()
mask = torch.randint(2, (4, 5, 3), dtype=torch.bool)
m = masked_tensor(data, mask)
print(m)

masked_tensor(
  [
    [
      [  1.5492,       --,       --],
      [  1.1851,       --,       --],
      [      --,       --,       --],
      [      --,  -0.9791,  -4.8729],
      [ -1.5334,       --,       --]
    ],
    [
      [      --,       --,  -1.6627],
      [      --,       --,   6.5170],
      [      --,  -0.6445,   0.4041],
      [      --,       --,       --],
      [ -4.0375,       --,  -4.3084]
    ],
    [
      [      --, -11.1796,  -6.1867],
      [  2.9545,   2.4948,       --],
      [  6.0095,   2.7039,  -3.1751],
      [ -7.1340,       --,   2.6322],
      [  4.3789,       --,       --]
    ],
    [
      [      --,   3.5293,       --],
      [  4.1633,  -7.1259,       --],
      [      --,       --,   0.7090],
      [      --,       --,  -0.9133],
      [ -8.6796,       --,   7.4433]
    ]
  ]
)


Example of indexing and advanced indexing

In [9]:
print(m[0])
print(m[torch.tensor([0, 2])])
print(m[m.mask()])

masked_tensor(
  [
    [  1.5492,       --,       --],
    [  1.1851,       --,       --],
    [      --,       --,       --],
    [      --,  -0.9791,  -4.8729],
    [ -1.5334,       --,       --]
  ]
)
masked_tensor(
  [
    [
      [  1.5492,       --,       --],
      [  1.1851,       --,       --],
      [      --,       --,       --],
      [      --,  -0.9791,  -4.8729],
      [ -1.5334,       --,       --]
    ],
    [
      [      --, -11.1796,  -6.1867],
      [  2.9545,   2.4948,       --],
      [  6.0095,   2.7039,  -3.1751],
      [ -7.1340,       --,   2.6322],
      [  4.3789,       --,       --]
    ]
  ]
)
masked_tensor(
  [  1.5492,   1.1851,  -0.9791,  -4.8729,  -1.5334,  -1.6627,   6.5170,  -0.6445,   0.4041,  -4.0375,  -4.3084, -11.1796,  -6.1867,   2.9545,   2.4948,   6.0095,   2.7039,  -3.1751,  -7.1340,   2.6322,   4.3789,   3.5293,   4.1633,  -7.1259,   0.7090,  -0.9133,  -8.6796,   7.4433]
)


### MaskedTensor gradient examples

In [10]:
torch.manual_seed(22)
# Sum needs custom autograd, since the mask of the input should be maintained
data = torch.randn(2, 2, 3).mul(5).float()
mask = torch.randint(2, (2, 2, 3), dtype=torch.bool)
m = masked_tensor(data, mask, requires_grad=True)
print(m)
s = torch.sum(m)
print("s: ", s)
s.backward()
print("m.grad: ", m.grad)

# sum needs to return a scalar MaskedTensor because the input might be fully masked
data = torch.randn(2, 2, 3).mul(5).float()
mask = torch.zeros(2, 2, 3, dtype=torch.bool)
m = masked_tensor(data, mask, requires_grad=True)
print("\n", m)
s = torch.sum(m)
print("s: ", s)
s.backward()
print("m.grad: ", m.grad)

masked_tensor(
  [
    [
      [      --,       --,  -0.5084],
      [  6.7935, -15.3725,       --]
    ],
    [
      [      --,   1.2078,       --],
      [  6.5820,       --,  -1.6679]
    ]
  ]
)
s:  masked_tensor( -2.9655, True)
m.grad:  masked_tensor(
  [
    [
      [      --,       --,   1.0000],
      [  1.0000,   1.0000,       --]
    ],
    [
      [      --,   1.0000,       --],
      [  1.0000,       --,   1.0000]
    ]
  ]
)

 masked_tensor(
  [
    [
      [      --,       --,       --],
      [      --,       --,       --]
    ],
    [
      [      --,       --,       --],
      [      --,       --,       --]
    ]
  ]
)
s:  masked_tensor(--, False)
m.grad:  masked_tensor(
  [
    [
      [      --,       --,       --],
      [      --,       --,       --]
    ],
    [
      [      --,       --,       --],
      [      --,       --,       --]
    ]
  ]
)


In [11]:
# Grad of multiplication of MaskedTensor and Tensor
x = masked_tensor(torch.tensor([3.0, 4.0]), torch.tensor([True, False]), requires_grad=True)
print("x:\n", x)
y = torch.tensor([2., 1.]).requires_grad_()
print("y:\n", y)
# The mask broadcast in the sense that the result is masked.
# In general a MaskedTensor is considered a generalization of Tensor's shape.
# The mask is a more complex, higher dimensional shape and thus the Tensor
# broadcasts to it. I'd love to find a more rigorous definition of this.
z = x * y
print("x * y:\n", z)
z.sum().backward()
print("\nx.grad: ", x.grad)
# The regular torch.Tensor now has a MaskedTensor grad
print("y.grad: ", y.grad)

x:
 masked_tensor(
  [  3.0000,       --]
)
y:
 tensor([2., 1.], requires_grad=True)
x * y:
 masked_tensor(
  [  6.0000,       --]
)

x.grad:  masked_tensor(
  [  2.0000,       --]
)
y.grad:  masked_tensor(
  [  3.0000,       --]
)
